In [3]:
import nltk

# Commented out IPython magic to ensure Python compatibility.
# %cd /content/ease/ease

## IMPORTS ##
from essay_set import EssaySet

from feature_extractor import FeatureExtractor
from predictor_set import PredictorSet
from predictor_extractor import PredictorExtractor
from sklearn.svm import SVR
import pickle
import pandas as pd
import csv

import numpy as np
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import learning_curve,GridSearchCV
#from sklearn import svm, grid_search

import nltk
#nltk.download('punkt')

#nltk.download('averaged_perceptron_tagger')

# The following 3 functions have been taken from Ben Hamner's github repository
# https://github.com/benhamner/Metrics
def Cmatrix(rater_a, rater_b, min_rating=None, max_rating=None):
    """
    Returns the confusion matrix between rater's ratings
    """
    assert(len(rater_a) == len(rater_b))
    if min_rating is None:
        min_rating = min(rater_a + rater_b)
    if max_rating is None:
        max_rating = max(rater_a + rater_b)
    num_ratings = int(max_rating - min_rating + 1)
    conf_mat = [[0 for i in range(num_ratings)]
                for j in range(num_ratings)]
    for a, b in zip(rater_a, rater_b):
        conf_mat[a - min_rating][b - min_rating] += 1
    return conf_mat


def histogram(ratings, min_rating=None, max_rating=None):
    """
    Returns the counts of each type of rating that a rater made
    """
    if min_rating is None:
        min_rating = min(ratings)
    if max_rating is None:
        max_rating = max(ratings)
    num_ratings = int(max_rating - min_rating + 1)
    hist_ratings = [0 for x in range(num_ratings)]
    for r in ratings:
        hist_ratings[r - min_rating] += 1
    return hist_ratings


def quadratic_weighted_kappa(y, y_pred):
    """
    Calculates the quadratic weighted kappa
    axquadratic_weighted_kappa calculates the quadratic weighted kappa
    value, which is a measure of inter-rater agreement between two raters
    that provide discrete numeric ratings.  Potential values range from -1
    (representing complete disagreement) to 1 (representing complete
    agreement).  A kappa value of 0 is expected if all agreement is due to
    chance.
    quadratic_weighted_kappa(rater_a, rater_b), where rater_a and rater_b
    each correspond to a list of integer ratings.  These lists must have the
    same length.
    The ratings should be integers, and it is assumed that they contain
    the complete range of possible ratings.
    quadratic_weighted_kappa(X, min_rating, max_rating), where min_rating
    is the minimum possible rating, and max_rating is the maximum possible
    rating
    """
    rater_a = y
    rater_b = y_pred
    min_rating=None
    max_rating=None
    rater_a = np.array(rater_a, dtype=int)
    rater_b = np.array(rater_b, dtype=int)
    assert(len(rater_a) == len(rater_b))
    if min_rating is None:
        min_rating = min(min(rater_a), min(rater_b))
    if max_rating is None:
        max_rating = max(max(rater_a), max(rater_b))
    conf_mat = Cmatrix(rater_a, rater_b,
                                min_rating, max_rating)
    num_ratings = len(conf_mat)
    num_scored_items = float(len(rater_a))

    hist_rater_a = histogram(rater_a, min_rating, max_rating)
    hist_rater_b = histogram(rater_b, min_rating, max_rating)

    numerator = 0.0
    denominator = 0.0

    for i in range(num_ratings):
        for j in range(num_ratings):
            expected_count = (hist_rater_a[i] * hist_rater_b[j]
                              / num_scored_items)
            d = pow(i - j, 2.0) / pow(num_ratings - 1, 2.0)
            numerator += d * conf_mat[i][j] / num_scored_items
            denominator += d * expected_count / num_scored_items

    return (1.0 - numerator / denominator)

In [24]:
train_set = pd.read_csv('/home/mehar/github/ease/aes_data/essay1/fold_0/train.txt', sep='\t')
x = train_set.to_numpy()
tester = x.tolist()
print(len(tester))
essaylist = []
scorelist = []
for i in range(0, len(tester)):
    z = tester[i]
    y = z[0].split(', ', 1)
#     #print(y)
    scorelist.append(float(y[0]))
    essaylist.append(y[1])

train = EssaySet()
print("Done1")
for i in range(0, len(essaylist)):
    train.add_essay(essaylist[i], scorelist[i])

print("Done2")
features=FeatureExtractor()
features.initialize_dictionaries(train)
X = features.gen_feats(train)
print("features train", X)
print("Done3")


1069
Done1
Done2
features train [[1.014e+03 2.170e+02 0.000e+00 ... 0.000e+00 0.000e+00 0.000e+00]
 [1.853e+03 3.930e+02 1.800e+01 ... 0.000e+00 6.000e+00 1.000e+00]
 [8.800e+02 1.820e+02 3.000e+00 ... 0.000e+00 0.000e+00 0.000e+00]
 ...
 [1.696e+03 3.280e+02 1.800e+01 ... 0.000e+00 0.000e+00 0.000e+00]
 [2.413e+03 5.130e+02 2.700e+01 ... 0.000e+00 1.000e+00 0.000e+00]
 [1.129e+03 2.370e+02 3.000e+00 ... 0.000e+00 0.000e+00 0.000e+00]]
Done3


In [7]:
test_score_set = pd.read_csv('scores/test6.csv', encoding = 'utf=8')
print(test_score_set.head())
print(len(test_score_set))
x_2 = test_score_set.to_numpy()
tester_2 = x_2.tolist()

test_scorelist = []

for i in range(0, len(tester_2)):
    z = tester_2[i]
    test_scorelist.append(float(z[0]))
# test_scorelist

   ORG_SCORE
0          2
1          1
2          3
3          4
4          3
360


In [27]:
import os 
import statistics
file_path = "AES_FinalTestcases/prompt1/"
files = os.listdir(file_path)
print(len(files), files)

67 ['svo_triplets_random_prompt1.csv', 'ut_beg_unbound_1.csv', 'ut_mid_bound_1.csv', 'songs_test_end_prompt_1.csv', 'wiki_topic_beg_unbound_1.csv', 'speeches_test_end_prompt_1.csv', 'wiki_end_unbound_1.csv', 'random_repeat_middle_1_prompt_1.csv', 'uf_mid_bound_1.csv', 'entities_beg_unbound_1.csv', 'uf_mid_unbound_1.csv', 'end_escalation_1.csv', 'random_repeat_introduction_1_prompt_1.csv', 'end_escalation_2.csv', 'wiki_mid_bound_1.csv', 'uf_end_bound_1.csv', 'incorrect_grammar_1.csv', 'contractions_aes_prompt1.csv', 'end_escalation_3.csv', 'end_escalation_4.csv', 'wiki_rel_mid_unbound_1.csv', 'entities_beg_bound_1.csv', 'entities_mid_bound_1.csv', 'wiki_rel_beg_bound_1.csv', 'start_escalation_2.csv', 'wiki_rel_end_bound_1.csv', 'speeches_test_beg_prompt_1.csv', 'random_repeat_middle_3_prompt_1.csv', 'entities_end_unbound_1.csv', 'uf_end_unbound_1.csv', 'wiki_rel_mid_bound_1.csv', 'ut_beg_bound_1.csv', 'babel_prompt1.csv', 'wiki_beg_bound_1.csv', 'start_escalation_1.csv', 'test8.csv', 'w

In [31]:
import os 
import statistics
import csv 

file_path = "AES_FinalTestcases/prompt1/"
files = os.listdir(file_path)
# print(files)
for testcase in files:
#     print(testcase)
    test_set = pd.read_csv(file_path+testcase, encoding="latin1")
    x = test_set.to_numpy()
    tester = x.tolist()
    test_essaylist = []

    for i in range(0, len(tester)):
        z = tester[i]
        if len(z) >1:
#             print(len(z))
    #         print(z[2])
#         y = z[0].split(', ', 1)
            test_essaylist.append(z[2])
        elif len(z) ==1:
#             print(z[1])
            test_essaylist.append(z)
    
#     print(len(test_essaylist), len(test_scorelist))
    test = EssaySet(essaytype="test")
    smaller = min(len(test_essaylist), len(test_scorelist))
    
    for i in range(0, smaller):
        test.add_essay(test_essaylist[i], test_scorelist[i])
    Y = features.gen_feats(test)
    print("features test", Y)
#     print("Done4")

    ## SCALING
    scaled_train = []
    for i in range(0, len(scorelist)):
        scaled_train.append(float((np.clip((scorelist[i]), a_min=2, a_max=12)/12)))

#     print("start training and prediciton")

    # print(scaled_train)
    from sklearn.svm import SVR

    # Values for prompt1
    Cs = [1]
    gammas = [0.00001]
#     Cs = [1, 10, 100, 1000]
#     gammas = [0.001, 0.0001, 0.000001]
# param_grid = {'C': Cs, 'gamma' : gammas}
# clf = GridSearchCV(SVR(kernel='rbf'), param_grid, cv =5)
    param_grid = {'C': Cs, 'gamma' : gammas}
    clf = GridSearchCV(SVR(kernel='rbf'), param_grid, cv =5)
    clf.fit(X, scaled_train)
    print(clf.best_params_)

    final = clf.predict(Y)

    finals = (np.clip(final,a_min=0,a_max=1)*12)
#     print(finals)
    file_dir_scores = "final_scores/prompt1/"+testcase
    pd.DataFrame(finals).to_csv(file_dir_scores)
#     finals_list = finals.tolist()
#     finals_list.encode()
#     print(type(finals_list))
    print("DONE ADDED TO CSV: ", testcase)
#     print("FL", finals_list)
#     import csv
#     file_dir_scores = "final_scores/prompt1/"+testcase
#     with open(file_dir_scores, 'wb') as myfile:
#         wr = csv.writer(myfile, quoting=csv.QUOTE_ALL)
#         wr.writerow(finals_list)
#     print("DONE ADDED TO CSV: ", testcase)
#     test_scorelist[:len(finals)]
#     test_scorelist = test_scorelist[:len(test_essaylist)]
#     print(len(test_scorelist))
    
#     print(quadratic_weighted_kappa(test_scorelist,finals))
#     print("##############")
#     print(finals_list)
#     df_pred = pd.DataFrame(finals_list)
#     df_org = pd.DataFrame(test_scorelist)
#     print("pred", df_pred.head(5))
#     print("org", df_org.head(5))

#     df_org["diff"] = abs(df_org[0] - df_pred[0])
#     mean_diff = df_org["diff"].mean()
#     print(""+"{:.3f}".format(mean_diff))
    #     mean_diff = statistics.mean(finals_list)
#     print("Mean", statistics.mean(diff_list))
#     print(""+"{:.3f}".format(mean_diff))
#     diff_list =[]
#     for i in range(0, len(finals_list)):
#     #     print(finals_list[i], test_scorelist[i])
#         diff = abs(test_scorelist[i] - finals_list[i])
#         diff_list.append(diff)
# #     print("", statistics.mean(diff_list))
#     mean1 = statistics.mean(diff_list)
#     print(""+"{:.3f}".format(mean1))
#     sdev = statistics.stdev(finals_list)
#     print(""+"{:.3f}".format(sdev))
    
#     neg_list =[]
#     pos_list =[]
# #     print(len(finals_list))
# #     print(len(test_scorelist))
#     for i in range(0, len(finals_list)):
#     #     print(finals_list[i], test_scorelist[i])
#         diff = test_scorelist[i] - finals_list[i]
#         if diff < 0:
#             neg_list.append(diff)
#         elif diff >=0:
#             pos_list.append(diff)
# #     print(len(neg_list))
#     print(len(pos_list))
#     mean_diff = statistics.mean(diff_list)
#     print("Mean", statistics.mean(diff_list))
#     print(""+"{:.3f}".format(mean_diff))
        # print(len(finals_list), len(test_scorelist))



features test [[2453.  508.   16. ...    0.    6.    0.]
 [2634.  551.   30. ...    0.    6.    0.]
 [1890.  380.    7. ...    0.    6.    0.]
 ...
 [2094.  393.    6. ...    0.    0.    0.]
 [2094.  393.    6. ...    0.    0.    0.]
 [2088.  393.    6. ...    0.    0.    0.]]
{'C': 1, 'gamma': 1e-05}
DONE ADDED TO CSV:  svo_triplets_random_prompt1.csv
features test [[18.  4.  0. ...  0.  0.  0.]
 [18.  4.  0. ...  0.  0.  0.]
 [18.  4.  0. ...  0.  0.  0.]
 ...
 [18.  4.  0. ...  0.  0.  0.]
 [18.  4.  0. ...  0.  0.  0.]
 [18.  4.  0. ...  0.  0.  0.]]
{'C': 1, 'gamma': 1e-05}
DONE ADDED TO CSV:  ut_beg_unbound_1.csv
features test [[18.  4.  0. ...  0.  0.  0.]
 [18.  4.  0. ...  0.  0.  0.]
 [18.  4.  0. ...  0.  0.  0.]
 ...
 [18.  4.  0. ...  0.  0.  0.]
 [18.  4.  0. ...  0.  0.  0.]
 [18.  4.  0. ...  0.  0.  0.]]
{'C': 1, 'gamma': 1e-05}
DONE ADDED TO CSV:  ut_mid_bound_1.csv
features test [[18.  4.  0. ...  0.  0.  0.]
 [18.  4.  0. ...  0.  0.  0.]
 [18.  4.  0. ...  0.  0.  

KeyboardInterrupt: 